<p>Содержание:</p>
<p>1  Описание работы</p>
<p>2  Загрузка необходимых библиотек</p>
<p>3  Загрузка, анализ и предобработка данных</p>
<p>3.1  Загрузка excel данных</p>
<p>3.2  Обработка целевых меток</p>
<p>3.3  Обработка дубликатов</p>
<p>3.4  Загрузка субтитров и словаря</p>
<p>3.5  Обработка пропусков</p>
<p>3.6  Оценка дисбаланса классов</p>
<p>4  Препроцессинг данных</p>
<p>4.1  Разбивка данных на обучающую и тестовую выборки</p>
<p>4.2  Векторизация</p>
<p>5  Обучение модели MultinomialNB</p>
<p>5.1  Выбор метрики качества</p>
<p>5.2  Построение pipiline и обучение</p>
<p>5.3  Оценка модели на тестовой (отложенной) выборке</p>
<p>5.4  Выводы по результатам работы модели</p>
<p>5.5  Обучение модели MultinomialNB на всех данных. Сохранение модели.</p>
<p>6  Выводы и рекомендации</p>
<p>Описание работыv
<p>Проект:</p>
<p>Запрос сформирован тем, что просмотр фильмов на оригинальном языке - это популярный и действенный метод упражнений по изучению иностранных языков. Важно выбрать фильм, который подходит студенту по уровню сложности, т.е. студент понимал 50-70 % диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.</p>

<p>Заказчику необходимо:

<p>разработать ML решение для автоматического определения уровня сложности англоязычных фильмов;
развернуть приложение для демонтрации работы ML решения (заказчик допустил возможность использваоние пакета Streamlit).</p>

<p>Исходные данные:

<p>размеченный датасет с названиями фильмов в формате excel, субтитрами и меткой уровня сложности языка (A1/A2/B1/B2/C1/C2). Предварительно данные в таблице уже обработаны (добавлены недостающие фильмы);</p>
<p>файлы субтитров в формате .srt, отсортированные по каталогам в соответствии с уровнем сложности. Предварительно все субтитры были перенесены в общую папку для 'D:/English_films_level/Datasets/Subtitles загрузки';</p>
<p>словари Oxford (на 3000 и 5000 тыс.слов), в которых слова на английском сгруппированны по уровню сложности.</p>
<p>План работы:</p>

<p>загрузка необходимых библиотек;</p>
<p>загрузка и ознакомление с данными;</p>
<p>предобработка данных (очистка от дубликатов, проверка наличия разметки для обучающих данных, определение исходного количества представленных данных, очитска текста субтитров, разбивка на обучающую и тестовую выборки);</p>
<p>препроцессинг данных (преобразование данных для обучения модели);</p>
<p>определение метрики качества;</p>
<p>обучение модели с побором гиперпараметров;</p>
<p>оценка модели на тестовой выборке;</p>
<p>развертывание модели для демонтрации работы с использованием библиотеки Streamlit;</p>
<p>рекомендации к улучшению проекта.</p>

## Загрузка необходимых библиотек

In [856]:
# установим недостающие пакеты
!pip install pysrt
!pip install spacy
!pip install en_core_web_sm
!python -m spacy download en_core_web_sm

# импортируем общие библиотеки
import os
import numpy as np               
import pandas as pd              
import pysrt                     
import spacy                     
import re
import warnings 
import matplotlib.pyplot as plt

# импортируем библиотеки sklearn 
from sklearn.datasets import load_files  
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import ColumnTransformer


     ---------------------------------------- 12.8/12.8 MB 6.7 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Загрузка excel данных

In [857]:
# сразу зададим константу случайного состояния для повторяемости результатов  
RAND_ST = 777

In [858]:
# del df
# загрузим данные из excel таблицы
df = pd.read_excel('D:/English_films_level/Datasets/movies_labels.xlsx', index_col='id')


In [859]:
df.head(10)

,Movie,Level
id,,
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+
5,All_dogs_go_to_heaven(1989),A2/A2+
6,An_American_tail(1986),A2/A2+
7,Babe(1995),A2/A2+
8,Back_to_the_future(1985),A2/A2+


In [860]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 240
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   241 non-null    object
 1   Level   241 non-null    object
dtypes: object(2)
memory usage: 5.6+ KB


Таблица excel содержит 241 размеченных фильмов/сериалов.

## Обработка целевых меток

In [861]:
# определим какие целевые метки содержат данные 
df['Level'].unique()

array(['B1', 'B2', 'A2/A2+', 'C1', 'B1, B2', 'A2/A2+, B1', 'A2'],
      dtype=object)

В данных присутствуют множественные метки для одного и того же фильма (например 'A2/A2+'). Зададим для таких фильмов единственную метку, соответсвующую максимальному значению уровня сложности.

Закодируем метки числовыми пордяками от 1 до 4.

Так же отмечено отсутствие фильмов с меткой A1, соответствующих самому легкому уровню сложности. После знакомства с представленными фильмами можно заметить, что таким фильмам (точнее мультикам) присвоена метка A2 (например для мультика 'Toy_story(1995)' присвоена метка A2, несмотря на то что многими специалистами уровень сложности определен как A1). В итоге предлагается придерживаться исходных размеченных данных, представленных Заказчиком и исключить из предсказания метку A1 и считать что она объединена с меткой A2.

Важное отступление: нельзя исключать субъективность разметки данных в фильмах. Каждый специалист может немного по-разному определять уровень сложности понимания английского. В таком случае можно ожидать смещения в предсказании любой модели, так как субъективность часто вносит неустранимую ошибку в предиктивность модели. С большой долей вероятности модель будет допускать ошибки в предсказании. Особенно на граничных значениях меток (например ошибаться в присвоении метки A2 или B1, B1 или B2). Такие ошибки впринципе допустимы в отношении поставленной задачи, так как позволяют добиться того, чтобы студент понимал 50-70 % диалогов даже при ошибочном предсказании модели в сторону присвоения более низкой метки, так как различия между такими классами менее существенны.

In [862]:
# определим словарь меток с кодированием значений
label_dict = {'A2': 1,
              'A2/A2+': 1,
              'B1': 2,
              'A2/A2+, B1': 2,
              'B2': 3,
              'B1, B2': 3,
              'C1': 4}
# заменим метки числовыми значениями
df = df.replace(label_dict)

## Обработка дубликатов

In [863]:
# проверим данные на наличие дубликатов
df[df.duplicated(keep=False)]

,Movie,Level
id,,
38,Powder(1995),2
43,Inside_out(2015),2
44,Inside_out(2015),2
68,Powder(1995),2
83,The_terminal(2004),2
99,The_terminal(2004),2


В данных присутствует 3 дубликата. Удалим их.

In [864]:
# удалим дубликаты 
df = df.drop_duplicates()
df.shape

(238, 2)

## Загрузка субтитров и словаря

Далее загрузим субтитры к фильмам из папки '/Datasets/Subtitles'.

In [865]:
# загрузим в список имена файлов из папки с субтитрами
films_name = os.listdir(path= 'D:/English_films_level/Datasets/Subtitles/')
# определим количество названий фильмов
print(f'Количество названий фильмов в папке с субтитрами: {len(films_name)}')

Количество названий фильмов в папке с субтитрами: 115


In [866]:
# проверим для скольких фильмов, имеющих метку из таблицы, предоставлены субтитры 
films_filtr = set(films_name) & set(df['Movie'] + '.srt')
print(f'Количество фильмов, имеющих метку и субтитры: {len(films_filtr)}')

Количество фильмов, имеющих метку и субтитры: 106


Далее загрузка субтитров в датафрейм будет осуществлена с применением функции очистки текста. Так же будет загружен словарь Classic Oxford для дальнейшего подсчета количества уникальных лемм, содержащихся в каждом тексте субтитров согласно уровню сложности.

In [867]:
# del american_oxford
oxford = load_files('D:/English_films_level/Datasets/Oxford/Classic Oxford/', shuffle=False, encoding='utf-8-sig')

In [868]:
oxford.data[0][:30]

'a, an \r\nabout \r\nabove \r\nacross'

In [876]:
# зададим регулярнеы выражения для очистки текста
HTML = re.compile(r'<.*?>') # html тэги меняем на пробел
TAG = r'{.*?}' # тэги меняем на пробел
COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]' # комменты в скобках меняем на пробел
LETTERS = r'[^a-zA-Z\.,!? ]' # все что не буквы меняем на пробел 
SPACES = r'([ ])\1+' # повторяющиеся пробелы меняем на один пробел
DOTS = r'[\.]+' # многоточие меняем на точку
# SYMB = r"[^\w\d'\s]" # знаки препинания кроме апострофа

# напишем функцию для очистки субтитров
def clean_subs(subs):
    subs = subs[1:] # удаляем первый рекламный субтитр
    txt = re.sub(HTML, ' ', subs.text) # html тэги меняем на пробел
    txt = re.sub(COMMENTS, ' ', txt) # комменты в скобках меняем на пробел
    txt = re.sub(LETTERS, ' ', txt) # все что не буквы меняем на пробел
    txt = re.sub(DOTS, r'.', txt) # многоточие меняем на точку
    txt = re.sub(SPACES, r'\1', txt) # повторяющиеся пробелы меняем на один пробел
    # txt = re.sub(SYMB, '', txt) # знаки препинания кроме апострофа на пустую строку
    txt = re.sub('www', '', txt) # кое-где остаётся www, то же меняем на пустую строку
    txt = txt.lstrip() # обрезка пробелов слева
    txt = txt.encode('ascii', 'ignore').decode() # удаляем все что не ascii символы   
    txt = txt.lower() # текст в нижний регистр
    return txt

# функция возвращающая количество уникальных лемм
# в тексте субтитров по каждому уровню
def lemma_count(lemmas, oxf, cat):
    func_dict = {'A1': 0,
                 'A2': 1,
                 'B1': 2,
                 'B2': 3,
                 'C1': 4}
    level = func_dict[cat]
    oxf_word_list = oxf[level].split()
    words = [lemma for lemma in lemmas if lemma in oxf_word_list]

    return len(set(words))

# загрузим для каждого фильма субтитры с использованием библиотеки pysrt
for film in films_filtr:
    try: 
        subs = pysrt.open(f'D:/English_films_level/Datasets/Subtitles/{film}')
        
    except:
        subs = pysrt.open(f'D:/English_films_level/Datasets/Subtitles/{film}', encoding='iso-8859-1')
 
    # вызов функии для очистки текста
    cln_subs = clean_subs(subs)
    df.loc[df['Movie'] == film[:-4], 'subs'] = cln_subs
    
    # используем библиотеку spacy для лемматизации 
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(cln_subs)
    lemma_list = [token.lemma_ for token in doc]
    
    # в цикле по каждой метке запишем в датафрейм кол-во уникальных лемм
    for lvl in ['A1', 'A2', 'B1', 'B2', 'C1']:
        df.loc[df['Movie'] == film[:-4], lvl+'_lemma_cnt'] = lemma_count(lemma_list, oxford.data, lvl)
                   
    
# выведем первые 10 записей
df.head(10)

,Movie,Level,subs,A1_lemma_cnt,A2_lemma_cnt,B1_lemma_cnt,B2_lemma_cnt,C1_lemma_cnt
id,,,,,,,,
0,10_Cloverfield_lane(2016),2,,0.0,0.0,0.0,0.0,0.0
1,10_things_I_hate_about_you(1999),2,,0.0,0.0,0.0,0.0,0.0
2,A_knights_tale(2001),3,,0.0,0.0,0.0,0.0,0.0
3,A_star_is_born(2018),3,,0.0,0.0,0.0,0.0,0.0
4,Aladdin(1992),1,,0.0,0.0,0.0,0.0,0.0
5,All_dogs_go_to_heaven(1989),1,,0.0,0.0,0.0,0.0,0.0
6,An_American_tail(1986),1,,0.0,0.0,0.0,0.0,0.0
7,Babe(1995),1,,0.0,0.0,0.0,0.0,0.0
8,Back_to_the_future(1985),1,,0.0,0.0,0.0,0.0,0.0


## Обработка пропусков

In [ ]:
# найдем пропуски в данных после загрузки
df[df['subs'].isna()].sort_values('Movie')

In [ ]:
# выдедем отсоритрованный список фильмов  для которых отсутствую субтитры в папке `'/Datasets/Subtitles'`
sorted(set(df['Movie'] + '.srt') - set(films_name))

Фильмы, для которых в датафрейме отсутствуют субтитры, это те же фильмы, для которых действительно в папке отсутствуют файлы .srt (ошибка в загрузке данных исключается). Удалим их из датафрейма.

In [ ]:
# удалим фильмы с пропущенными субтитрами
df.dropna(inplace=True)
# проверим удаление
df[df['subs'].isna()]

## Оценка дисбаланса классов

Для классификации и определения метрики качества важно проверять данные на наличие дисбаланса целевых классов.

In [822]:
# определим количество фильмов, представленных в кажой категории сложности
df['Level'].value_counts()

2    39
3    37
1    25
4     6
Name: Level, dtype: int64

В данных присутствует сильный дисбаланс по целевым классам (меткам):

129 фильмов с уровнем сложности 3 (метка B2);
56 фильмов с уровнем сложности 2 (метка B1);
39 фильмов с уровнем сложности 4 (метка C1);
32 фильма с уровнем сложности 1 (метка A2).
Вывод: для борьбы с дисбалансом классов есть несколько методов, которые не являются универсальными по своей сути и каждый должен определяться исходя из требований задачи и исходных данных. Так undersampling (выравнивание выборки за счет снижения количества фильмов с мажоритарными классами) недопустим в текущих условиях и так малого количества исходных данных. В данном случае снижается общая вариативность данных, которая необходима модели машинного обучения для нахождения признаков и их связей с правильным предсказанием целевой метрики. В рамках решения задач NLP малое количество обучающих данных приводит к более 'скудному' словарю. Отсутствие 'богатства' словаря часто приводит к неверному предсказанию модели на новых текстах, которые состоят из слов не вошедших в 'vocabulary' обученной модели.

Таким образом в данном случае было бы уместнее применить upsampling (выравнивание выборки за счет увеличения методом дублирования фильмов с миноритарными классами). Но с другой стороны увеличение выборки за счет дублирования текста в миноритарных классах не приведет к увеличению словаря модели и как следствие не увеличит качество прогнозов. В таком случае предлагается оставить выборку без изменений.

## Препроцессинг данных

## Разбивка данных на обучающую и тестовую выборки

Разбивка данных будет произведена в соотношении обучающая/тестовая выборка - 80/20 %. Важным моментом который стоит учитывать - сохранение соотношения баланса классов в обеих выборках. Для этого будет использован параметр 'stratify'.

In [825]:
# разобъем данные на валидационную и тестовую выборки
df_train, df_test = train_test_split(df, random_state=RAND_ST, test_size=.2, stratify=df['Level'])
# определим независимые признаки и зависимую целевую метку
X_train = df_train.drop(['Level', 'Movie'], axis=1)
y_train = df_train['Level']
X_test = df_test.drop(['Level', 'Movie'], axis=1)
y_test = df_test['Level']
# определим размеры обучающей и тестовой выборки
print(f'Размер обучающей выборки: {X_train.shape, y_train.shape}')
print(f'Размер тестовой выборки: {X_test.shape, y_test.shape}')

Размер обучающей выборки: ((85, 6), (85,))
Размер тестовой выборки: ((22, 6), (22,))


## Векторизация

Векторизуем текст субтитров с помощью 'CountVectorizer' с гиперпараметром min_df=4, чтобы исключить специфические слова для определенных фильмов, которые не несут явной языковой нагрузки.

In [826]:
# векторизуем текст субтитров обучающей и тестовой выборки
vect = CountVectorizer(min_df=4).fit(X_train['subs'])
X_train_vect = vect.transform(X_train['subs'])
X_test_vect = vect.transform(X_test['subs'])

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
# выведем результаты (признаки) векторизации данных
feature_names = vect.get_feature_names_out()
print('Количество признаков: {}'.format(len(feature_names)))
print('Первые 20 признаков:\n{}'.format(feature_names[:20]))
print('Признаки с 2000 по 2060:\n{}'.format(feature_names[2000:2060]))
print('Каждый 100-й признак:\n{}'.format(feature_names[::100]))

## Обучение модели MultinomialNB

Для обучения модели на текстовых данных будет использована модель наивного байесовского классификатора MultinomialNB. Такая модель хорошо подходит для высокоразмерных данных, обучается довольно быстро при назначительно меньшем качестве в отличии от линейной модели.

Обучение модели будет проводится с использованием GridSearcCV для подбора гиперпараметра 'alpha' с кроссвалидацией на 5-и фолдах. Во избежание утечки данных будет использован Pipeline.

## Выбор метрики качества

В связи с тем, что:

в данных обнаружен дисбаланс классов;
в текущей задаче является важным как точность ('precision') так и полнота ('recall') предсказания модели;
лучшей метрикой для подгонки и оценки модели принимается средне-гармоническое точности и полноты - F1-мера. Для мультиклассификации будет использована взвешенная F1-мера (F1_weighted).

In [ ]:
# напишем функцию для построения матрицы ошибок
def plot_confusion_matrix(y_test, y_preds, model):
    fig, ax = plt.subplots(figsize=(16,10))
    cm = confusion_matrix(y_test, y_preds)
    cmp = ConfusionMatrixDisplay(cm, display_labels = model.classes_ )
    cmp.plot(ax=ax)
    plt.suptitle('Матрица ошибок', y=0.92)
    plt.show()

## Построение pipiline и обучение

In [ ]:
%%time

# выделим отдельную категорию признаков
other_colls = X_train.columns[1:]

# инициализируем вектор-объект
vector = CountVectorizer(min_df=4)

# создадим препроцессор для групп признаков
preprocessor = ColumnTransformer([
    ('other', 'passthrough', other_colls),
    ('txt', vector, 'subs')
])

# инициализируем модель MultinomialNB
classifier = MultinomialNB()

# создадим pipeline
pipe = Pipeline(steps=[('prep', preprocessor),
                       ('clf', classifier)])

# обучим модель с подбором гиперпараметра alpha на 5 фолдах
param_grid = {'clf__alpha': np.arange(0.001, 0.3, 0.002)}
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_weighted')
grid.fit(X_train, y_train)
# выведем лучший score и значение гиперпараметра
print(grid.best_score_)
print(grid.best_params_)

Лучшая модель показала результат - 0.658 (F1_weighted) при подобранном гиперпараметре 'alpha' равном 0.137.

Оценим модель на тестовой выборке.

## Оценка модели на тестовой (отложенной) выборке

In [ ]:
# получим предсказания на тестовой выборке
predictions = grid.predict(X_test)
# выведем раезльтаты оценки по метрике F1-weighted
print(f'Метрика F1-weighted на обучающей выборке:{f1_score(grid.predict(X_train), y_train, average="weighted")}')
print(f'Метрика F1-weighted на тестовой выборке:{f1_score(predictions, y_test, average="weighted")}')
print('-'*50)
# выведем классификационную таблицу и матрицу ошибок
print(classification_report(y_test, predictions))
plot_confusion_matrix(y_test, predictions, grid)

Лучшая модель показала результат - 0.63 (F1_weighted) на тестовой выборке при подобранном гиперпараметре 'alpha' равном 0.137.

## Выводы по результатам работы модели

Из оценки классификатора на тестовой выборке можно сделать следующие выводы:

как и ожидалось, модель чаще всего ошибается на граничных метках. Например модель часто назначает фильмам с меткой B2 метку B1. По условиям задачи где, студенты должны минимум понимать 50-70% диалогов, ошибка прогнозирования в сторону наименьшего класса допустима в пределах граничных классов;
хуже всего классифицированы фильмы с метками A1 и B1. В первом случае за счет низкого значения метрики recall (среди небольшого количества фильмов представленных в этом классе меток (7 фильмов), всего 2 фильма размечены верно). Во втором случае за счет низкого значения метрики precision (модель часто ошибается в присвоении метки B1, т.е. среди всех размеченных фильмов с присвоением метки B1 (21 фильм) только 8 размечены верно.
Использование дополнительно созданных признаков, с количеством уникальных слов из классического словаря Oxford в каждом фильме, не дали значительного прироста в качестве предсказаний, что может подтверждать предположение о субъективности разметки представленных Заказчиком фильмов, в силу предвзятости специалистов по английскому языку в отношении сложности диалогов.

В целом с учетом предположения о субъективности разметки представленных Заказчиком фильмов, а так же использования быстрого и простого наивного Байесовского классификатора, удалось достичь приемлемых результатов. Взвешенная метрика F1-weighted на тестовой выборке составила 0.63.

## Обучение модели MultinomialNB на всех данных. Сохранение модели.

In [ ]:
# обучим модель с подбором гиперпараметра alpha на 5 фолдах
# на всей выборке и оценим качество
X_full = pd.concat([X_train, X_test])
y_full = pd.concat([y_train, y_test])
param_grid = {'clf__alpha': np.arange(0.001, 0.3, 0.002)}
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_weighted')
grid.fit(X_full, y_full)
# выведем лучший score и значение гиперпараметра
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
# сохраним модель в папку models
dump(grid, './models/model_bayesNB.joblib')
# сохраним классический oxford словарь
dump(oxford.data, './oxford/classic_oxford.joblib')

## Выводы и рекомендации

В ходе работы было выполнено:

загрузка необходимых библиотек;
загрузка и ознакомление с данными (загружены субтитры, фильмы с метками уровня сложности, классический словарь Oxford с 5000 словами);
предобработка данных (очистка от дубликатов, проверка наличия разметки для обучающих данных, определение исходного количества представленных данных, очитска текста субтитров, разбивка на обучающую и тестовую выборки);
препроцессинг данных, а именно преобразование текство субтитров с компактную разреженную матрицу с помощью CountVectorizer с гиперпараметром min_df=4, чтобы исключить специфические слова для определенных фильмов, которые не несут явной языковой нагрузки.;
добавление дополнительных признаков- количество уникальных лемм (слов основной формы), содержащихся в каждом тексте субтитров согласно уровню сложности по классичесокму словарю Oxford;
обоснование использования метрики качества F1-weighted;
обучение модели-классификатора MultinomialNB с побором гиперпараметров на 5-и фолдах перекрестной прокерки;
оценка модели на тестовой выборке;
обучение модели на всей выборке с подбором гиперпарамтера на 5-и фолдах перекрестной прокерки
сохранение лучшей модели в формате 'joblib', а так же классического словаря Oxford для дальнейшего развертывания в демонстрационном приложении Streamlit.

При решении задачи обнаружены следующие проблеммы:

Отсутствуют фильмы с уровнем сложности A1;
В данных присутствует сильный дисбаланс по остальным классам (меткам):
129 фильмов с уровнем сложности 3 (метка B2);
56 фильмов с уровнем сложности 2 (метка B1);
39 фильмов с уровнем сложности 4 (метка C1);
32 фильма с уровнем сложности 1 (метка A2).
Наличие большой вероятности в субъективности разметки данных в фильмах. Каждый специалист может немного по-разному определять уровень сложности понимания английского. В таком случае можно ожидать смещения в предсказании любой модели, так как субъективность часто вносит неустранимую ошибку в предиктивность модели. С большой долей вероятности модель будет допускать ошибки в предсказании. Особенно на граничных значениях меток (например ошибаться в присвоении метки A2 или B1, B1 или B2). Такие ошибки впринципе допустимы в отношении поставленной задачи, так как позволяют добиться того, чтобы студент понимал 50-70 % диалогов даже при ошибочном предсказании модели в сторону присвоения более низкой метки, так как различия между такими классами менее существенны.

Выводы по результатам работы модели:

Лучшая модель показала результат - 0.63 (F1_weighted) на тестовой выборке при подобранном гиперпараметре 'alpha' равном 0.137.

как и ожидалось, модель чаще всего ошибается на граничных метках. Например модель часто назначает фильмам с меткой B2 метку B1. По условиям задачи где, студенты должны минимум понимать 50-70% диалогов, ошибка прогнозирования в сторону наименьшего класса допустима в пределах граничных классов;
хуже всего классифицированы фильмы с метками A1 и B1. В первом случае за счет низкого значения метрики recall (среди небольшого количества фильмов представленных в этом классе меток (7 фильмов), всего 2 фильма размечены верно). Во втором случае за счет низкого значения метрики precision (модель часто ошибается в присвоении метки B1, т.е. среди всех размеченных фильмов с присвоением метки B1 (21 фильм) только 8 размечены верно.
Использование дополнительно созданных признаков, с количеством уникальных слов из классического словаря Oxford в каждом фильме, не дали значительного прироста в качестве предсказаний, что может подтверждать предположение о субъективности разметки представленных Заказчиком фильмов, в силу предвзятости специалистов по английскому языку в отношении сложности диалогов.

В целом с учетом предположения о субъективности разметки представленных Заказчиком фильмов, а так же использования быстрого и простого наивного Байесовского классификатора, удалось достичь приемлемых результатов. Взвешенная метрика F1-weighted на модели, обученной на всех представленных данных, при кроссвалидации на 5-и фолдах показала результат в 0.68 по метрике F1-weighted.

Рекомендации:

Собрать больше фильмов с размеченным уровнем сложности с одинаковым количеством в каждом классе, включая уровень A1. В данном случае можно добиться лучшей пргоностической способности модели;
Разметка фильмов по уровню сложности должна быть лишена субъективности и опираться на общепринятые правила определения. Размеченные данные (фильмы) должны поступать от классифицированных специалистов по английскому языку, либо с общеизвесиных и общепринятых источников;
(Предпочтительная и важная рекомендация для Заказчика) Для создания модели предсказания уровня сложности понимания фильмов необходимо полностью исключить заранее размеченные данные.
В таком случае обучение модели будет основываться на других правилах, таких как:

классический и американский словари Oxford,
количество слов в предложениях,
скорость речи,
центральные оценки (среднее, медиана) количетства букв в слове в общем по всему тексту
и т.д.
Для такой разарботки модели потребуется дополнительное время (около 10 дней). В данном случае снижен риск субъективности в оценке сложности понимания английских фильмов, так как будут использованы более понятные и строгие правила определения.